In [39]:
import torch
from torch import nn, optim
from torch.nn import functional as F
from torch.utils.data import DataLoader
from torch.utils.data import TensorDataset
from torch.autograd import Variable
from sklearn.metrics import r2_score
from sklearn import preprocessing
#import scipy.io as sio

from models import VAE,AEBase
from models import DNN
import numpy as np
import pandas as pd
import models
import utils as ut

In [40]:
from scipy.stats import pearsonr

# Parameters

In [41]:
# Define parameters
epochs = 500 #200,500,1000  
#dim_au_in = 20049
dim_au_out = 512 #8, 16, 32, 64, 128, 256,512
dim_dnn_in = dim_au_out
dim_dnn_out=1
select_drug = 'Gefitinib'
na = -1

# Import data

In [42]:
data_r=pd.read_csv('data/GDSCexpression.csv',index_col=0)
label_r=pd.read_csv('data/GDSClabel.csv',index_col=0)

In [43]:
label_r=label_r.fillna(na)

In [44]:
hvg,adata = ut.highly_variable_genes(data_r)

Observation names are not unique. To make them unique, call `.obs_names_make_unique`.


In [45]:
label_r

,Erlotinib,AICAR,Camptothecin,Vinblastine,Cisplatin,Cytarabine,Docetaxel,Methotrexate,ATRA,Gefitinib,...,CMK,Pyrimethamine,JW-7-52-1,A-443654,GW843682X,MS-275,Parthenolide,MG-132,KIN001-135,TGX221
BxPC-3,-1.000000,0.159708,0.188393,0.436127,0.154274,0.113764,0.249987,0.015324,0.015324,0.020293,...,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000
KMOE-2,0.002108,0.339814,0.318206,0.281740,0.063808,0.140681,0.101530,0.191210,0.031229,0.007419,...,0.022662,0.025217,0.270791,0.095608,0.139914,0.171747,0.026670,0.013904,0.002116,0.000956
MFM-223,0.002148,0.334087,0.122671,0.361438,0.051983,0.186659,0.228797,0.003639,0.023043,0.003639,...,0.038997,0.006167,0.002185,0.102592,0.020338,0.010874,0.000970,0.035206,0.007051,0.000323
NUGC-3,-1.000000,0.095099,0.416792,0.422631,0.266409,0.237811,0.234952,0.011558,0.028838,0.153002,...,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000
OC-314,-1.000000,0.012740,0.479915,0.390145,0.067738,0.125869,0.249668,0.110418,0.002214,0.020309,...,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
KP-N-S19s,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,...,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000
BC-3,0.003515,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,...,0.048227,0.146476,0.400087,0.205247,0.147090,0.298803,0.023796,0.017219,0.002065,0.069181
Panc 08.13,-1.000000,0.152818,0.196279,0.180653,0.090963,0.093197,0.097267,0.018269,0.010237,0.009405,...,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000
EKVX,0.177238,0.078395,0.078938,0.072213,0.007225,0.003470,0.129184,0.095642,0.111598,0.126364,...,0.020472,0.044047,0.243635,0.079375,0.019859,0.177759,0.014843,0.114431,0.007216,0.063911


In [46]:
selected_idx = label_r.loc[:,select_drug]!=na

In [47]:
data_r.columns = adata.var_names

# Your is gene-cell, mine is cell-gene

In [48]:
data = data_r.loc[selected_idx,hvg]

In [49]:
label = label_r.loc[selected_idx,select_drug]
scaler = preprocessing.StandardScaler(with_mean=True, with_std=True)
data = scaler.fit_transform(data)


In [50]:
print(np.std(data))
print(np.mean(data))

0.9999999999999999
-9.065550491767716e-18


In [51]:
data.shape

(675, 3462)

In [52]:
label_r.shape

(789, 139)

# Split test train

In [53]:
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(data, label, test_size=0.2, random_state=42)

In [54]:
print(data.shape)
print(label.shape)
print(X_train.shape, Y_train.shape)
print(X_test.shape, Y_test.shape)

(675, 3462)
(675,)
(540, 3462) (540,)
(135, 3462) (135,)


In [55]:
print(X_train.max())
print(X_train.min())

19.375997920629374
-7.911806694002141


# AE MODEL

In [56]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
# Assuming that we are on a CUDA machine, this should print a CUDA device:
print(device)
torch.cuda.set_device(device)

cuda:0


# Add all data to AE

In [57]:
trainData = torch.FloatTensor(X_train).to(device)
testData = torch.FloatTensor(X_test).to(device)
y = torch.FloatTensor(Y_train.values).to(device)
allData = torch.FloatTensor(data).to(device)

# construct TensorDataset
train_dataset = TensorDataset(trainData, trainData)
test_dataset = TensorDataset(testData, testData)
all_dataset = TensorDataset(allData, allData)

trainDataLoader1 = DataLoader(dataset=train_dataset, batch_size=200, shuffle=False)
trainDataLoaderall = DataLoader(dataset=all_dataset, batch_size=200, shuffle=False)

In [58]:
dataloader = trainDataLoaderall

# The model

In [59]:
model = AEBase(input_dim=data.shape[1],latent_dim=256,hidden_dims=[1024,512,256])

In [60]:
print(model)

AEBase(
  (encoder): Sequential(
    (0): Sequential(
      (0): Linear(in_features=3462, out_features=1024, bias=True)
      (1): BatchNorm1d(1024, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): LeakyReLU(negative_slope=0.01)
      (3): Dropout(p=0.3, inplace=False)
    )
    (1): Sequential(
      (0): Linear(in_features=1024, out_features=512, bias=True)
      (1): BatchNorm1d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): LeakyReLU(negative_slope=0.01)
      (3): Dropout(p=0.3, inplace=False)
    )
    (2): Sequential(
      (0): Linear(in_features=512, out_features=256, bias=True)
      (1): BatchNorm1d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): LeakyReLU(negative_slope=0.01)
      (3): Dropout(p=0.3, inplace=False)
    )
  )
  (bottleneck): Linear(in_features=256, out_features=256, bias=True)
  (decoder_input): Linear(in_features=256, out_features=256, bias=True)
  (decoder): Sequent

In [61]:
#model = VAE(dim_au_in=data_r.shape[1],dim_au_out=128)
if torch.cuda.is_available():
    model.cuda()
model.to(device)
optimizer = optim.Adam(model.parameters(), lr=1e-4)
loss_function = nn.SmoothL1Loss()

In [62]:
loss_train = np.zeros((epochs, 1))

# Train autoencoder

for epoch in range(epochs):
    # 不需要label，所以用一个占位符"_"代替
    for batchidx, (x, _) in enumerate(trainDataLoaderall):
        x.requires_grad_(True)
        # encode and decode 
        output = model(x)
        # compute loss
        print(output.shape)
        loss = loss_function(output, x)      
        # update
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
           
    loss_train[epoch,0] = loss.item()  
    print('Epoch: %04d, Training loss=%.8f' %
          (epoch+1, loss.item()))


torch.Size([200, 3462])
torch.Size([200, 3462])
torch.Size([200, 3462])
torch.Size([75, 3462])
Epoch: 0001, Training loss=0.40740085
torch.Size([200, 3462])
torch.Size([200, 3462])
torch.Size([200, 3462])
torch.Size([75, 3462])
Epoch: 0002, Training loss=0.39197275
torch.Size([200, 3462])
torch.Size([200, 3462])
torch.Size([200, 3462])
torch.Size([75, 3462])
Epoch: 0003, Training loss=0.37923217
torch.Size([200, 3462])
torch.Size([200, 3462])
torch.Size([200, 3462])
torch.Size([75, 3462])
Epoch: 0004, Training loss=0.36773765
torch.Size([200, 3462])
torch.Size([200, 3462])
torch.Size([200, 3462])
torch.Size([75, 3462])
Epoch: 0005, Training loss=0.35750961
torch.Size([200, 3462])
torch.Size([200, 3462])
torch.Size([200, 3462])
torch.Size([75, 3462])
Epoch: 0006, Training loss=0.34957319
torch.Size([200, 3462])
torch.Size([200, 3462])
torch.Size([200, 3462])
torch.Size([75, 3462])
Epoch: 0007, Training loss=0.34184936
torch.Size([200, 3462])
torch.Size([200, 3462])
torch.Size([200, 3462

Epoch: 0062, Training loss=0.26449719
torch.Size([200, 3462])
torch.Size([200, 3462])
torch.Size([200, 3462])
torch.Size([75, 3462])
Epoch: 0063, Training loss=0.26388493
torch.Size([200, 3462])
torch.Size([200, 3462])
torch.Size([200, 3462])
torch.Size([75, 3462])
Epoch: 0064, Training loss=0.26359504
torch.Size([200, 3462])
torch.Size([200, 3462])
torch.Size([200, 3462])
torch.Size([75, 3462])
Epoch: 0065, Training loss=0.26331189
torch.Size([200, 3462])
torch.Size([200, 3462])
torch.Size([200, 3462])
torch.Size([75, 3462])
Epoch: 0066, Training loss=0.26254642
torch.Size([200, 3462])
torch.Size([200, 3462])
torch.Size([200, 3462])
torch.Size([75, 3462])
Epoch: 0067, Training loss=0.26234102
torch.Size([200, 3462])
torch.Size([200, 3462])
torch.Size([200, 3462])
torch.Size([75, 3462])
Epoch: 0068, Training loss=0.26198936
torch.Size([200, 3462])
torch.Size([200, 3462])
torch.Size([200, 3462])
torch.Size([75, 3462])
Epoch: 0069, Training loss=0.26090786
torch.Size([200, 3462])
torch.S

torch.Size([75, 3462])
Epoch: 0125, Training loss=0.23905975
torch.Size([200, 3462])
torch.Size([200, 3462])
torch.Size([200, 3462])
torch.Size([75, 3462])
Epoch: 0126, Training loss=0.23817959
torch.Size([200, 3462])
torch.Size([200, 3462])
torch.Size([200, 3462])
torch.Size([75, 3462])
Epoch: 0127, Training loss=0.23750272
torch.Size([200, 3462])
torch.Size([200, 3462])
torch.Size([200, 3462])
torch.Size([75, 3462])
Epoch: 0128, Training loss=0.23770466
torch.Size([200, 3462])
torch.Size([200, 3462])
torch.Size([200, 3462])
torch.Size([75, 3462])
Epoch: 0129, Training loss=0.23807044
torch.Size([200, 3462])
torch.Size([200, 3462])
torch.Size([200, 3462])
torch.Size([75, 3462])
Epoch: 0130, Training loss=0.23866501
torch.Size([200, 3462])
torch.Size([200, 3462])
torch.Size([200, 3462])
torch.Size([75, 3462])
Epoch: 0131, Training loss=0.23640449
torch.Size([200, 3462])
torch.Size([200, 3462])
torch.Size([200, 3462])
torch.Size([75, 3462])
Epoch: 0132, Training loss=0.23767094
torch.Si

torch.Size([75, 3462])
Epoch: 0187, Training loss=0.22415900
torch.Size([200, 3462])
torch.Size([200, 3462])
torch.Size([200, 3462])
torch.Size([75, 3462])
Epoch: 0188, Training loss=0.22567067
torch.Size([200, 3462])
torch.Size([200, 3462])
torch.Size([200, 3462])
torch.Size([75, 3462])
Epoch: 0189, Training loss=0.22426218
torch.Size([200, 3462])
torch.Size([200, 3462])
torch.Size([200, 3462])
torch.Size([75, 3462])
Epoch: 0190, Training loss=0.22470404
torch.Size([200, 3462])
torch.Size([200, 3462])
torch.Size([200, 3462])
torch.Size([75, 3462])
Epoch: 0191, Training loss=0.22235702
torch.Size([200, 3462])
torch.Size([200, 3462])
torch.Size([200, 3462])
torch.Size([75, 3462])
Epoch: 0192, Training loss=0.22378992
torch.Size([200, 3462])
torch.Size([200, 3462])
torch.Size([200, 3462])
torch.Size([75, 3462])
Epoch: 0193, Training loss=0.22283903
torch.Size([200, 3462])
torch.Size([200, 3462])
torch.Size([200, 3462])
torch.Size([75, 3462])
Epoch: 0194, Training loss=0.22132033
torch.Si

torch.Size([75, 3462])
Epoch: 0249, Training loss=0.21246813
torch.Size([200, 3462])
torch.Size([200, 3462])
torch.Size([200, 3462])
torch.Size([75, 3462])
Epoch: 0250, Training loss=0.21295239
torch.Size([200, 3462])
torch.Size([200, 3462])
torch.Size([200, 3462])
torch.Size([75, 3462])
Epoch: 0251, Training loss=0.21252964
torch.Size([200, 3462])
torch.Size([200, 3462])
torch.Size([200, 3462])
torch.Size([75, 3462])
Epoch: 0252, Training loss=0.21108465
torch.Size([200, 3462])
torch.Size([200, 3462])
torch.Size([200, 3462])
torch.Size([75, 3462])
Epoch: 0253, Training loss=0.21109332
torch.Size([200, 3462])
torch.Size([200, 3462])
torch.Size([200, 3462])
torch.Size([75, 3462])
Epoch: 0254, Training loss=0.21320766
torch.Size([200, 3462])
torch.Size([200, 3462])
torch.Size([200, 3462])
torch.Size([75, 3462])
Epoch: 0255, Training loss=0.20965697
torch.Size([200, 3462])
torch.Size([200, 3462])
torch.Size([200, 3462])
torch.Size([75, 3462])
Epoch: 0256, Training loss=0.21043050
torch.Si

torch.Size([200, 3462])
torch.Size([200, 3462])
torch.Size([75, 3462])
Epoch: 0312, Training loss=0.19971353
torch.Size([200, 3462])
torch.Size([200, 3462])
torch.Size([200, 3462])
torch.Size([75, 3462])
Epoch: 0313, Training loss=0.19933626
torch.Size([200, 3462])
torch.Size([200, 3462])
torch.Size([200, 3462])
torch.Size([75, 3462])
Epoch: 0314, Training loss=0.20088661
torch.Size([200, 3462])
torch.Size([200, 3462])
torch.Size([200, 3462])
torch.Size([75, 3462])
Epoch: 0315, Training loss=0.20193540
torch.Size([200, 3462])
torch.Size([200, 3462])
torch.Size([200, 3462])
torch.Size([75, 3462])
Epoch: 0316, Training loss=0.19954519
torch.Size([200, 3462])
torch.Size([200, 3462])
torch.Size([200, 3462])
torch.Size([75, 3462])
Epoch: 0317, Training loss=0.20122343
torch.Size([200, 3462])
torch.Size([200, 3462])
torch.Size([200, 3462])
torch.Size([75, 3462])
Epoch: 0318, Training loss=0.19860698
torch.Size([200, 3462])
torch.Size([200, 3462])
torch.Size([200, 3462])
torch.Size([75, 3462]

Epoch: 0374, Training loss=0.19166002
torch.Size([200, 3462])
torch.Size([200, 3462])
torch.Size([200, 3462])
torch.Size([75, 3462])
Epoch: 0375, Training loss=0.19114691
torch.Size([200, 3462])
torch.Size([200, 3462])
torch.Size([200, 3462])
torch.Size([75, 3462])
Epoch: 0376, Training loss=0.18914108
torch.Size([200, 3462])
torch.Size([200, 3462])
torch.Size([200, 3462])
torch.Size([75, 3462])
Epoch: 0377, Training loss=0.19072828
torch.Size([200, 3462])
torch.Size([200, 3462])
torch.Size([200, 3462])
torch.Size([75, 3462])
Epoch: 0378, Training loss=0.19013917
torch.Size([200, 3462])
torch.Size([200, 3462])
torch.Size([200, 3462])
torch.Size([75, 3462])
Epoch: 0379, Training loss=0.18917264
torch.Size([200, 3462])
torch.Size([200, 3462])
torch.Size([200, 3462])
torch.Size([75, 3462])
Epoch: 0380, Training loss=0.19045407
torch.Size([200, 3462])
torch.Size([200, 3462])
torch.Size([200, 3462])
torch.Size([75, 3462])
Epoch: 0381, Training loss=0.18849675
torch.Size([200, 3462])
torch.S

torch.Size([200, 3462])
torch.Size([75, 3462])
Epoch: 0437, Training loss=0.18299472
torch.Size([200, 3462])
torch.Size([200, 3462])
torch.Size([200, 3462])
torch.Size([75, 3462])
Epoch: 0438, Training loss=0.18226419
torch.Size([200, 3462])
torch.Size([200, 3462])
torch.Size([200, 3462])
torch.Size([75, 3462])
Epoch: 0439, Training loss=0.18136422
torch.Size([200, 3462])
torch.Size([200, 3462])
torch.Size([200, 3462])
torch.Size([75, 3462])
Epoch: 0440, Training loss=0.18172385
torch.Size([200, 3462])
torch.Size([200, 3462])
torch.Size([200, 3462])
torch.Size([75, 3462])
Epoch: 0441, Training loss=0.18081677
torch.Size([200, 3462])
torch.Size([200, 3462])
torch.Size([200, 3462])
torch.Size([75, 3462])
Epoch: 0442, Training loss=0.18016757
torch.Size([200, 3462])
torch.Size([200, 3462])
torch.Size([200, 3462])
torch.Size([75, 3462])
Epoch: 0443, Training loss=0.18076363
torch.Size([200, 3462])
torch.Size([200, 3462])
torch.Size([200, 3462])
torch.Size([75, 3462])
Epoch: 0444, Training 

Epoch: 0498, Training loss=0.17408587
torch.Size([200, 3462])
torch.Size([200, 3462])
torch.Size([200, 3462])
torch.Size([75, 3462])
Epoch: 0499, Training loss=0.17454344
torch.Size([200, 3462])
torch.Size([200, 3462])
torch.Size([200, 3462])
torch.Size([75, 3462])
Epoch: 0500, Training loss=0.17398685


In [76]:
save(model.state_dict(), 'saved/models/ae.pth')

''saved/models/ae.pth')' was not found in history, as a file, url, nor in the user namespace.


In [64]:
recon_batch = model(trainData)

In [65]:
loss_function(recon_batch,trainData)

tensor(0.2155, device='cuda:0', grad_fn=<SmoothL1LossBackward>)

In [66]:
train_embeddings = model.encode(trainData)

In [67]:
feature = train_embeddings.cpu().detach().numpy()

In [68]:
from sklearn import linear_model
clf = linear_model.Lasso(alpha=0.001)
clf.fit(feature, Y_train.values)

Lasso(alpha=0.001)

In [69]:
from sklearn.ensemble import RandomForestRegressor
RFrg = RandomForestRegressor(random_state=0, n_estimators=100)
RFrg.fit(feature, Y_train.values)

RandomForestRegressor(random_state=0)

In [70]:
testFeature = model.encode(testData)
lasso = clf.predict(testFeature.detach().cpu().numpy())

In [71]:
rfresult = RFrg.predict(testFeature.detach().cpu().numpy())

In [72]:
r2_score(lasso,Y_test)

-5.07412733740147

In [73]:
r2_score(rfresult,Y_test)

-3.8667001867915856

In [74]:
Y_test.values

array([0.010893, 0.003969, 0.126364, 0.04815 , 0.013301, 0.003975,
       0.006658, 0.236411, 0.024986, 0.112077, 0.004105, 0.007248,
       0.015496, 0.079042, 0.066385, 0.006305, 0.006078, 0.025742,
       0.030093, 0.024503, 0.011125, 0.037697, 0.012029, 0.003817,
       0.010667, 0.067386, 0.057037, 0.006329, 0.067224, 0.003564,
       0.186092, 0.108377, 0.101198, 0.029871, 0.02189 , 0.043306,
       0.12798 , 0.005638, 0.038664, 0.002193, 0.005611, 0.157488,
       0.007132, 0.006106, 0.008437, 0.031169, 0.054449, 0.006736,
       0.016306, 0.003624, 0.093631, 0.011592, 0.010451, 0.003869,
       0.011615, 0.189386, 0.003639, 0.023794, 0.020825, 0.036415,
       0.00255 , 0.005367, 0.018914, 0.013027, 0.185878, 0.003549,
       0.004383, 0.007288, 0.012238, 0.019928, 0.075604, 0.01131 ,
       0.006595, 0.00253 , 0.005594, 0.018333, 0.015419, 0.010771,
       0.008096, 0.03138 , 0.043866, 0.014501, 0.002152, 0.015858,
       0.142685, 0.134998, 0.006991, 0.049048, 0.033088, 0.005

In [75]:
rfresult

array([0.04544396, 0.06734093, 0.06377047, 0.02340427, 0.03321962,
       0.03108255, 0.04430354, 0.08361159, 0.085765  , 0.07418036,
       0.01756385, 0.04774304, 0.02550264, 0.02741602, 0.05130644,
       0.02950179, 0.0285139 , 0.03134224, 0.03029437, 0.02092909,
       0.01270665, 0.02905975, 0.03553276, 0.05071436, 0.0337828 ,
       0.04824643, 0.07182165, 0.02350863, 0.04646199, 0.02535163,
       0.03639966, 0.05288374, 0.09163362, 0.09985648, 0.04434686,
       0.03048915, 0.04460238, 0.01941024, 0.02975804, 0.03917842,
       0.0538914 , 0.03354143, 0.03364884, 0.01611548, 0.0274551 ,
       0.02108319, 0.01297739, 0.05200652, 0.03115727, 0.02409462,
       0.05304045, 0.1028788 , 0.03304246, 0.01428098, 0.01918673,
       0.06179765, 0.02147982, 0.02919494, 0.01954293, 0.0367334 ,
       0.05657205, 0.04624026, 0.03179698, 0.03727589, 0.09738907,
       0.01895641, 0.03559419, 0.01458342, 0.02372696, 0.04588088,
       0.09081751, 0.02690984, 0.06742653, 0.02153077, 0.04761

In [ ]:
from sklearn.manifold import TSNE
tsne = TSNE(n_components=2, verbose=1, perplexity=40, n_iter=300)
tsne_results = tsne.fit_transform(feature)

In [ ]:
tsne_results

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

plt.scatter(tsne_results[:, 0], tsne_results[:, 1], label="label")
plt.legend()
plt.savefig("saved/figures/tsne_vae_gdsc.png")

In [ ]:
EPOCH = 500

In [ ]:
# Load data
# data type conversion
B_feature = torch.FloatTensor(feature).to(device)
y = torch.FloatTensor(Y_train.values).to(device)
# construct TensorDataset
b_data = TensorDataset(B_feature, y)
trainDataLoader2 = DataLoader(dataset=b_data, batch_size=200, shuffle=False)

In [ ]:
# Initialization DNN model

predictor = DNN(128, dim_dnn_out).to(device)
optimizer = optim.Adam(predictor.parameters(), lr=1e-3,betas=(0.9,0.99))
#loss1-softmax
loss_func = nn.MSELoss().to(device)
#loss2-sigmoid
#loss_func = nn.BCELoss()
#loss3-sigmoid
#loss_func = nn.CrossEntropyLoss()

#criterion = torch.nn.MSELoss(size_average=True)
#criterion = torch.nn.BCELoss(size_average=True) # Defined loss function
#optimizer = optim.Adm(model.parameters(), lr=0.01) # Defined optimizer
loss_train = np.zeros((epochs, 1))

In [ ]:
# train model
for epoch in range(EPOCH):
    print('Epoch: ',epoch)
    for step,(batch_x,batch_y) in enumerate(trainDataLoader2):
        b_x = Variable(batch_x)
        b_y = Variable(batch_y)
        # predict label
        output = predictor(b_x)
        # b_y=F.sigmoid(b_y) 
        
        #print
        #print(output)
        #print(b_y)
        # compute loss
        loss = loss_func(output,b_y)
        #loss = criterion(output, b_y)
        
        # update
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    loss_train[epoch,0] = loss.item()  
    print('Epoch: %04d, Training loss=%.8f' %
          (epoch+1, loss.item())) 

# Save model
torch.save(predictor.state_dict(), 'saved/models/DNN_GDSC.pkl')

In [ ]:
testpredict = predictor(testFeature)

In [ ]:
testpredict

In [ ]:
from sklearn.metrics import mean_squared_error

In [ ]:
r2_score(testpredict.detach().cpu().numpy(),Y_test)

In [ ]:
mean_squared_error(testpredict.detach().cpu().numpy(),Y_test)